In [15]:
import pandas as pd
import sys
import esig
import numpy as np
import random
import matplotlib.pyplot as plt
import sklearn.metrics
import itertools
import os.path
import time
import zipfile
import iisignature
from joblib import Memory, Parallel, delayed
from datetime import datetime
from signal import signal
from tqdm import tqdm
from functools import lru_cache
from scipy.stats import chi2
from sklearn.metrics import roc_auc_score
from sklearn.covariance import EmpiricalCovariance, MinCovDet
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

from dummy_data_signatures_v2.py import normalise_stream
from dummy_data_signatures_v2.py import calculate_signatures
from dummy_data_signatures_v2.py import paths

from mah_outlier_calc import mah_distance

ImportError: cannot import name 'paths_calc' from 'dummy_data_signatures_v2' (C:\Users\dmitr\Desktop\project work\dummy_data_signatures_v2.py)

In [7]:
clean_events = pd.read_csv('C:\\Users\\dmitr\\Desktop\\project work\\dummy data\\clean_data.csv')
anomalous_events_labels = pd.read_csv('C:\\Users\\dmitr\\Desktop\\project work\\dummy data\\data_anomaly_labels.csv')
anomalous_events_timestamps = set(anomalous_events_labels['timestamp'].values.tolist())
events = pd.read_csv('C:\\Users\\dmitr\\Desktop\\project work\\dummy data\\data.csv')

In [8]:
day = 86400
observed = [np.int_(0), np.int_(1), np.int_(2), np.int_(3), np.int_(4)]
depth = 2
time_variable = True
daytime_variable = True
stream_length = 7 * day
stream_spacing = 2 * day

no_extrapollation = False
lead_lag = False

In [9]:
streams = paths(events, anomalous_events_timestamps, observed,  stream_length, stream_spacing, time_variable, daytime_variable, no_extrapollation, lead_lag)
corpus = paths(clean_events, [], observed,  stream_length, stream_spacing, time_variable, daytime_variable, no_extrapollation, lead_lag)


AttributeError: module 'dummy_data_signatures_v2' has no attribute 'split_stream'

In [ ]:
norm_streams = map(lambda stream: normalise_stream(stream, time_variable = True, daytime_variable = True, scale_data = False, preprocess_data_columnwise = False), streams)
norm_corpus = map(lambda stream: normalise_stream(stream, time_variable = True, daytime_variable = True, scale_data = False, preprocess_data_columnwise = False), corpus)


In [ ]:
streams_signatures = calculate_signatures(norm_streams, depth)
corpus_signatures = calculate_signatures(norm_corpus, depth)

In [ ]:
mah_distance(signatures)